### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
7013,2023-11-17,Brasil Nbb,20:00,Caxias do Sul,São José,2.14,1.69,149.5,1.88,1.79,1.5,1.97,1.74
7014,2023-11-17,Brasil Nbb,20:00,Corinthians Paulista,Paulistano,1.73,2.06,146.5,1.88,1.88,-2.5,1.98,1.74
7015,2023-11-17,Brasil Nbb,20:00,Pinheiros,São Paulo,2.32,1.59,157.5,1.86,1.88,2.5,1.97,1.75
7016,2023-11-17,Eua Nba,00:00,Golden State Warriors,Oklahoma City Thunder,2.37,1.69,225.5,1.83,1.93,-4.5,1.90,1.90
7017,2023-11-17,Eua Nba,21:00,Washington Wizards,New York Knicks,3.35,1.40,226.5,1.80,2.01,5.5,2.08,1.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7166,2023-11-17,Turquia Tb2L,11:00,Ege Universitesi,Istanbul Efendileri,2.59,1.45,142.5,1.83,1.83,3.5,2.07,1.67
7167,2023-11-17,Uruguai Liga Uruguaia,21:00,Goes,Malvin,1.90,1.80,165.5,1.87,1.79,-1.5,2.09,1.66
7168,2023-11-17,Uruguai Liga Uruguaia,21:00,Larre Borges,Hebraica y Macabi,2.70,1.42,161.5,1.87,1.79,4.5,2.02,1.71
7169,2023-11-17,Uruguai Liga Uruguaia,21:00,Trouville,Bigua,1.78,1.93,167.5,1.83,1.83,-2.5,2.08,1.66


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
8,22:00,Eua Nba,New Orleans Pelicans,Denver Nuggets,222.5,1.81,0.9999,Over
10,16:00,Europa Euroliga,Lyon-Villeurbanne,Baskonia,164.5,1.91,0.7272,Over
85,13:00,Eua Ncaa,UMBC Retrievers,Sacred Heart,157.5,1.91,0.8383,Over
86,13:30,Eua Ncaa,LSU,North Texas,129.5,1.91,0.8529,Over
99,19:45,Eua Ncaa,Kent State,Hampton,155.5,1.91,0.8682,Over
119,21:00,Eua Ncaa,Pittsburgh,Jacksonville,140.5,1.91,0.8770,Over
123,21:00,Eua Ncaa,UMass,Harvard,153.5,1.91,0.8270,Over
136,22:00,Eua Ncaa,Western Illinois,Southern,141.5,1.91,0.8493,Over
140,23:00,Eua Ncaa,Ole Miss,Sam Houston St.,133.5,1.91,0.8599,Over


### Over v2 (NB)

In [9]:
# Duplicando o dataset
df_over_v2 = jogos.copy()

df_over_v2.reset_index(drop=True, inplace=True)
df_over_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v2.empty:
    mdl_over_v2 = load_model('ML/over/v2/v2_over', verbose=False)
    prev = predict_model(mdl_over_v2, df_over_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Last_CG_H'] >= 0) & (prev['Last_CG_H'] < 130)) | ((prev['Last_CG_H'] >= 183) & (prev['Last_CG_H'] < 1485))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V2 (NB) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V2 (NB) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
3,00:00,Eua Nba,Golden State Warriors,Oklahoma City Thunder,225.5,1.83,0.6008,Over
5,21:30,Eua Nba,Cleveland Cavaliers,Detroit Pistons,221.5,1.92,0.9768,Over
6,21:30,Eua Nba,San Antonio Spurs,Sacramento Kings,232.5,1.82,0.6218,Over
9,15:00,Europa Euroliga,Zalgiris Kaunas,Bayern,157.5,1.91,0.7278,Over
10,16:00,Europa Euroliga,Lyon-Villeurbanne,Baskonia,164.5,1.91,0.7401,Over
11,16:15,Europa Euroliga,Panathinaikos,Virtus Bologna,161.5,1.87,0.9189,Over
12,16:30,Europa Euroliga,Barcelona,Valencia,156.5,1.91,0.7532,Over
13,16:45,Europa Euroliga,Partizan,Alba Berlin,168.5,1.91,0.7301,Over
22,21:00,Argentina Liga A,Argentino,Platense,156.5,1.88,0.5336,Over
23,22:00,Argentina Liga A,Quimsa,Obera TC,162.5,1.87,0.5046,Over


### Over v3 (NB)

In [10]:
# Duplicando o dataset
df_over_v3 = jogos.copy()

df_over_v3.reset_index(drop=True, inplace=True)
df_over_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v3.empty:
    mdl_over_v3 = load_model('ML/over/v3/v3_over', verbose=False)
    prev = predict_model(mdl_over_v3, df_over_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V3 (NB) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V3 (NB) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
5,21:30,Eua Nba,Cleveland Cavaliers,Detroit Pistons,221.5,1.92,0.8152,Over
6,21:30,Eua Nba,San Antonio Spurs,Sacramento Kings,232.5,1.82,0.5336,Over
7,22:00,Eua Nba,Chicago Bulls,Orlando Magic,215.5,1.85,0.6657,Over
9,15:00,Europa Euroliga,Zalgiris Kaunas,Bayern,157.5,1.91,0.7955,Over
10,16:00,Europa Euroliga,Lyon-Villeurbanne,Baskonia,164.5,1.91,0.8064,Over
...,...,...,...,...,...,...,...,...
139,23:00,Eua Ncaa,Colorado State,UMKC,143.5,1.91,0.8782,Over
140,23:00,Eua Ncaa,Ole Miss,Sam Houston St.,133.5,1.91,0.7944,Over
141,23:00,Eua Ncaa,Oregon,Tennessee State,151.5,1.91,0.8852,Over
143,23:00,Eua Ncaa,Stanford,East. Washington,160.5,1.91,0.8171,Over
